# Traitement de texte et NLP

In [2]:
# on importe les differentes librairies
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import spacy
import networkx as nx
import nltk
import scipy

# on importe les different module pour le traitement de texte
from gensim.utils import simple_preprocess
from wordcloud import WordCloud
from collections import Counter
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# on importe notre bdd
avis_norm = pd.read_csv("BDD/avis_norm.csv")

In [13]:
avis_norm.head(3)

,author,date_published,title_review,note,title,url,comment,comment_propre,nombre_mots_comment_propre
0,Monsieur Guillaume,2021-01,Voyages sur les ailes des papillons,0.8,Mariposas,https://www.trictrac.net/jeu-de-societe/maripo...,"Lorsque le jeu est jeu, bon, réflexif, joli po...",jeu jeu bon réflexif joli sensible style illus...,33
1,morlockbob,2020-10,le festival de Michoacan,0.7,Mariposas,https://www.trictrac.net/jeu-de-societe/maripo...,Comment continuer après un mega hit ? Simpleme...,mega simplement oiseau papillon jeu principe f...,43
2,SwatSh,2021-02,Vivez la migration des monarques,0.7,Mariposas,https://www.trictrac.net/jeu-de-societe/maripo...,"Vin d'jeu: Avec Mariposas, Elizabeth Hargrave ...",vin jeu mariposa hargrave épopée contrainte fi...,33


In [4]:
# Charger le modèle de langue française
nlp = spacy.load('fr_core_news_sm')
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop

# Assurer que tous les commentaires sont des chaînes de caractères et gérer les valeurs nulles
avis_norm = pd.read_csv("BDD/avis_norm.csv")
avis_norm['comment'] = avis_norm['comment'].fillna('')  # Remplacer les valeurs NaN par des chaînes vides
avis_norm['comment'] = avis_norm['comment'].astype(str)  # Convertir tout en chaîne de caractères

def filtrer_commentaire(commentaire):
    doc = nlp(commentaire)
    pos_exclues = ['DET', 'CONJ', 'PRON', 'ADP', 'CCONJ', 'PUNCT', 'VERB']
    mots_filtres = [token.lemma_ for token in doc if token.pos_ not in pos_exclues and token.lemma_ not in fr_stop]
    return ' '.join(mots_filtres)



In [6]:
# Obtenir des statistiques descriptives pour les longueurs de commentaires
statistiques_longueurs = avis_norm['nombre_mots_comment_propre'].describe()

# Afficher les statistiques
print(statistiques_longueurs)

count    125299.000000
mean         29.728904
std          36.155198
min           0.000000
25%           8.000000
50%          21.000000
75%          40.000000
max         985.000000
Name: nombre_mots_comment_propre, dtype: float64


In [7]:
# Filtrage des commentaires longs en utilisant un seuil spécifique basé sur le nombre de mots
seuil_nombre_mots = 40 # Considéré comme long s'il contient plus de 40 mots 25% des commentaires plus long
commentaires_longs = avis_norm[avis_norm['nombre_mots_comment_propre'] > seuil_nombre_mots]['comment_propre'].tolist()

# nombre de commentaires longs
print(f"Nombre de commentaires longs : {len(commentaires_longs)}")

for i in range(0,10):
    print(f"\ncommentaires {i} \n")
    print(commentaires_longs[i])
    

Nombre de commentaires longs : 30273

commentaires 0 

mega simplement oiseau papillon jeu principe familial papillon carte pays fleur reproduir point ville carte couleur difficile pose jeton jeu objectif saison point départ papillon adulte transformation force force objectif masse base jeu hasardeux moment tirage jeton fastidieux répétitif mécanique beaucoup original wingspan

commentaires 1 

mariposa beau grand plateau petit papillon plein fleur jeton nla bien thème naissance papillon naucun party désagréable vraiment choix carte tactique point objectif point mexique collection carte nla jeu familial contrairement wingpan créateur option joueur expérimenté vite objectif changer dilemme nl jeu vite répétitif ncurieusement livret règle bien jeu catégorie accessible public an point simple relecture exemple papillon double naissance papillon bien dramatique dommage interaction inexistant bien plateau nla chance présent tirage carte particulier dernier manche maximum point score différen

In [8]:
# Joindre tous les commentaires longs en une seule chaîne de texte
texte_long = " ".join(commentaires_longs)

# Créer un objet WordCloud
nuage_de_mots = WordCloud(background_color='white', max_words=100, contour_color='steelblue')

# Générer le nuage de mots
nuage_de_mots.generate(texte_long)

# Afficher le nuage de mots avec matplotlib
plt.figure(figsize=(10, 7))
plt.imshow(nuage_de_mots, interpolation='bilinear')
plt.axis('off')  # Ne pas afficher les axes pour une meilleure visualisation
plt.show()

In [9]:
# Calculer les 100 mots les plus fréquents dans la chaîne de texte longue
mots_frequents = Counter(texte_long.split()).most_common(100)
for f in mots_frequents:
    print(f)

('jeu', 136586)
('bien', 39065)
('carte', 38679)
('joueur', 36198)
('bon', 28825)
('partie', 24436)
('qu', 20492)
('petit', 18514)
('party', 17527)
('point', 17033)
('règle', 16772)
('tour', 15188)
('trop', 13913)
('vraiment', 12357)
('grand', 11643)
('simple', 11611)
('action', 10369)
('plateau', 10215)
('temps', 10105)
('matériel', 10053)
('beaucoup', 9897)
('fois', 9671)
('thème', 9498)
('coup', 9289)
('non', 8618)
('faire', 8289)
('dé', 8089)
('choix', 7272)
('hasard', 7101)
('fin', 7076)
('gros', 6805)
('tuile', 6793)
('plaisir', 6705)
('stratégie', 6682)
('mécanique', 6599)
('chose', 6454)
('mal', 6271)
('long', 6261)
('vite', 6256)
('nombre', 6166)
('bref', 6156)
('extension', 5928)
('adversaire', 5774)
('mécanisme', 5662)
('qualité', 5641)
('moment', 5618)
('ambiance', 5614)
('part', 5601)
('système', 5230)
('autant', 5200)
('monde', 5126)
('mieux', 5090)
('côté', 5006)
('place', 4926)
('personnage', 4773)
('intéressant', 4686)
('dernier', 4679)
('main', 4676)
('base', 4640)
('

**Analyse de co-occurrence et réseaux de mots / N-grammes**

In [10]:
# Tokenisation
tokens = word_tokenize(texte_long, language='french')

# Génération des bigrammes
bigrammes = list(ngrams(tokens, 2))

# Comptage des bigrammes les plus fréquents
freq_bigrammes = Counter(bigrammes)
print(freq_bigrammes.most_common(15))

[(('bon', 'jeu'), 4974), (('jeu', 'bien'), 3345), (('petit', 'jeu'), 3072), (('jeu', 'carte'), 2951), (('jeu', 'jeu'), 2678), (('fin', 'partie'), 2306), (('règle', 'simple'), 2289), (('tour', 'jeu'), 2098), (('jeu', 'bon'), 2066), (('bien', 'jeu'), 2065), (('point', 'victoire'), 2039), (('jeu', 'simple'), 1763), (('jeu', 'joueur'), 1751), (('jeu', 'gestion'), 1681), (('jeu', 'base'), 1678)]


**Comparer le vocabulaire entre les différents utilisateurs.**

In [11]:

# Assurez-vous d'abord de remplacer les valeurs NaN par une chaîne vide ou par un placeholder approprié
df_grouped = avis_norm.fillna('')

# Ensuite, regroupez les commentaires par auteur et les concaténez en un seul texte, en s'assurant que tout est traité comme une chaîne
df_grouped = avis_norm.groupby('author')['comment_propre'].apply(lambda x: ' '.join(x.astype(str))).reset_index()

# Limiter à 100 auteurs si nécessaire (vous avez mentionné 10 dans la demande initiale, mais utilisé 100 ici)
df_limited = df_grouped.head(100)

# Charger les stop words en français
stop_words_french = stopwords.words('french')

# Prétraitement et vectorisation TF-IDF avec les stop words français de nltk
vectorizer = TfidfVectorizer(stop_words=stop_words_french)
X = vectorizer.fit_transform(df_grouped['comment_propre'])

# Le reste du code reste inchangé
knn = NearestNeighbors(n_neighbors=4, metric='cosine')
knn.fit(X)

# Trouver les auteurs les plus proches pour chaque auteur, en ignorant le premier résultat
distances, indices = knn.kneighbors(X)

for i in range(len(df_grouped)):
    print(f"Auteurs les plus similaires à {df_grouped.iloc[i]['author']}:")
    for j in range(1, 4):  # Commence à 1 pour ignorer l'auto-recommandation et imprime les 3 suivants
        if j < len(df_grouped):  # Vérifie si l'indice est dans la plage
            print(f"- {df_grouped.iloc[indices[i, j]]['author']} avec une distance de {distances[i, j]:.2f}")
    print()

Auteurs les plus similaires à *FitzChevalerie*:
- Pac avec une distance de 0.49
- Robbenblues avec une distance de 0.49
- redeagle avec une distance de 0.50

Auteurs les plus similaires à --- JD ---:
- kakawette avec une distance de 0.67
- romjé avec une distance de 0.67
- LudiKev avec une distance de 0.69

Auteurs les plus similaires à -Yo-:
- dugsyu avec une distance de 1.00
- dumoncel avec une distance de 1.00
- duinhir avec une distance de 1.00

Auteurs les plus similaires à 20.100:
- limp avec une distance de 0.28
- daguiseb avec une distance de 0.28
- lynkowsky avec une distance de 0.29

Auteurs les plus similaires à 20cil:
- Pso76 avec une distance de 0.41
- limp avec une distance de 0.42
- momov avec une distance de 0.43

Auteurs les plus similaires à 26vince:
- dugsyu avec une distance de 1.00
- dumoncel avec une distance de 1.00
- duinhir avec une distance de 1.00

Auteurs les plus similaires à 3dman:
- magicjl avec une distance de 0.46
- BONY avec une distance de 0.46
- Marc

In [12]:
# Filtrer les commentaires de l'auteur 'Adouskar' et afficher les colonnes 'title_review' et 'comment'
print("Les commentaires de 20.100:")
print(avis_norm.loc[avis_norm['author'] == '20.100', ['title_review', 'comment','note']])

# Filtrer les commentaires de l'auteur 'echow464' et afficher les colonnes 'title_review' et 'comment'
print("\nLes commentaires de JJL :")
print(avis_norm.loc[avis_norm['author'] == 'JJL', ['title_review', 'comment','note']])


Les commentaires de 20.100:
                                  title_review  \
21108                        Plumage et ramage   
23506                                 Parfait?   
26299               Addicitf pas dans le dico?   
26717                              Magnifique!   
31850            Une petite perfection ludique   
35914                       Traîne en longueur   
40888                   Et en plus c'est Zoli!   
53234                    Ton 34, tu le gardes!   
53621                     Nostalgie, nostalgie   
64946              Ben un effort d'imagination   
100685                              Consensuel   
104974              Ficelle, selle ... de mer?   
106128                     20 minutes pas plus   
112191                          Le cow-boy mou   
112530                       Basari plus mieux   
113590                        Ultime défouloir   
114040                        C'est du Knizia?   
114292         Attention, mes mongols arrivent   
114816        C'est fi